In [7]:
import copy
# TODO: 
#  5. Use Cuda
#  7. Add analog of batch norm (additive parameter in DestinationNeuron (need to make AdStep optimiser to support not-scalar values for that)
# what is "self.only_one_output_request" ???
# After that can think on Cifar10 convnet death

import random
import time

import numpy as np
import pandas as pd
import torch
torch.use_deterministic_algorithms(True)
device="cuda"
device="cpu"
torch.set_default_device(device)
import torch.nn as nn
import torch.utils.data
import importlib
from ai_libs.basic.deep_reload import deep_reload
import livenet
deep_reload(livenet)
import ai_libs.simple_log as simple_log
from ai_libs.simple_log import LOG
simple_log.level = simple_log.LogLevel.INFO
import math
from matplotlib import pyplot as plt
plt.ion()
%matplotlib TkAgg
livenet.backend.utils.set_seed()
var = 42
print(torch.__version__)
np.set_printoptions(precision=3)
%precision 4


2.5.1+cu124


'%.4f'

In [8]:
downscale = (4, 4)
test = livenet.datasets.get_cifar10_test()
train = livenet.datasets.get_cifar10_train()
test_x, test_y = livenet.datasets.to_plain(*test, downscale=downscale, to_gray=True, to_odd=False)
train_x, train_y = livenet.datasets.to_plain(*train, downscale=downscale, to_gray=True, to_odd=False)
test_x = test_x.to(device)
train_x = train_x.to(device)



Files already downloaded and verified
Files already downloaded and verified


In [9]:
network = livenet.nets.create_perceptron(train_x.shape[1], 8, 10)
batch_size = 1000
batch_iterator = livenet.gen_utils.batch_iterator(train_x, train_y, batch_size)
criterion = livenet.nets.criterion_classification_n
optimizer = livenet.nets.create_optimizer(network)
trainer = livenet.net_trainer.NetTrainer(network, batch_iterator, criterion, optimizer, epoch_size=len(train_x) // batch_size)

LiveNet


In [10]:
# simple_log.level = simple_log.LogLevel.DEBUG
#network.mortal = False
network.context.regularization_l1 = 0.001
optimizer.learning_rate = 0.01
trainer.step(1000)


Iˈ0.000 49 3.043+0.060reg params=524 av_abs_grad 0.005 av_abs_delta: 0.01 av_abs_output 0.05 max_abs_output 0.2 max_abs_param 0.7 hcut 1e-05 lcut 0.2 livenet/net_trainer.py:122
Iˈ6.460 99 2.873+0.074reg params=441 av_abs_grad 0.006 av_abs_delta: 0.01 av_abs_output 0.09 max_abs_output 0.4 max_abs_param 1.0 hcut 2e-05 lcut 0.2 livenet/net_trainer.py:122
Iˈ12.032 149 2.835+0.077reg params=392 av_abs_grad 0.006 av_abs_delta: 0.01 av_abs_output 0.1 max_abs_output 0.5 max_abs_param 1.1 hcut 2e-05 lcut 0.2 livenet/net_trainer.py:122
Iˈ17.025 199 2.816+0.079reg params=353 av_abs_grad 0.007 av_abs_delta: 0.01 av_abs_output 0.1 max_abs_output 0.6 max_abs_param 1.1 hcut 9e-06 lcut 0.2 livenet/net_trainer.py:122
Iˈ21.638 249 2.804+0.081reg params=331 av_abs_grad 0.007 av_abs_delta: 0.01 av_abs_output 0.1 max_abs_output 0.7 max_abs_param 1.2 hcut 8e-06 lcut 0.2 livenet/net_trainer.py:122
Iˈ26.020 299 2.798+0.081reg params=315 av_abs_grad 0.007 av_abs_delta: 0.01 av_abs_output 0.2 max_abs_output 0.8

In [5]:
import sys
def deep_getsizeof(obj, seen=None):
    """Recursively finds size of objects, including contents."""
    if seen is None:
        seen = set()

    obj_id = id(obj)
    if obj_id in seen:
        return 0  # Avoid double-counting the same object
    seen.add(obj_id)

    size = sys.getsizeof(obj)

    if isinstance(obj, dict):
        size += sum((deep_getsizeof(k, seen) + deep_getsizeof(v, seen)) for k, v in obj.items())
    elif isinstance(obj, (list, tuple, set, frozenset)):
        size += sum(deep_getsizeof(item, seen) for item in obj)

    return size

deep_getsizeof(network.context.life_stat)


56

In [6]:
pred = network(test_x)
pred_ind = np.argmax(pred.detach().numpy(), axis=1, keepdims=True)
diff = test_y.numpy() - pred_ind
f"accuracy {len(diff[diff == 0]) / len(diff)}"


'accuracy 0.3085'

In [117]:
print(network.context.topology_stat.get_stat())
for name, p in sorted(network.named_parameters(),key=lambda x: x[0]):
    print(name, f"{p.item():.2f}")


{'dangle': {'RegularNeuron': 0, 'DestinationNeuron': 0}, 'useless': {'RegularNeuron': 0, 'InputNeuron': 57}}
D0 0.39
D1 -0.43
I0->N1 0.85
I0->N3 -0.99
I11->N3 0.59
I19->N1 -0.73
I41->N1 -0.29
I44->N3 0.15
I46->N1 -0.41
I63->N3 0.25
N1 -0.07
N1->D0 -1.15
N1->D1 0.94
N3 0.04
N3->D0 -1.49
N3->D1 0.81


In [143]:
life_stat = network.context.life_stat
df = pd.DataFrame(life_stat)
df2 = df[(df["type"] == "parameter") & (df["name"] == "I12->N7")]
#df2 = df[(df["type"] == "parameter") & (df["name"] == "D0")]

#df2 = df[(df["type"] == "delta") & (df["name"] == "N0->D0")]
#df2 = df[(df["type"] == "delta") & (df["name"] == "D0")]
#df2 = df[(df["type"] == "gradient") & (df["name"] == "N0->D0")]
#df2 = df[(df["type"] == "gradient") & (df["name"] == "D0")]
#df2 = df[(df["type"] == "max_output") & (df["name"] == "D0")]
# df2 = df[(df["type"] == "output_low_cut_ratio") & (df["tick"] == 199)]
# df2 = df[(df["type"] == "parameter") ]
df2 = df[(df["type"] == "output_low_cut_ratio")]
gps = df2.groupby("tick")
res = []
for name, group in gps:
    res.append({"tick": name, "value": group["value"].min()})
df2 = pd.DataFrame(res)
df2["value"].plot(grid=True, use_index=False, style='.-')
# df2 = df[(df["type"] == "output_high_cut_ratio")]
#df2 = df2.groupby("tick", as_index=False).max()

<Axes: >

In [35]:

from livenet.net_trainer import get_summary_stat

get_summary_stat(pd.DataFrame(network.context.life_stat))

'av_abs_grad 0.006 av_abs_delta: 0.005 av_abs_output nan max_abs_param 1.2 hcut 0 lcut 0.4'

In [12]:
rng = random.Random(x=0)
d0 = 1000
d1 = 20000
d2 = 15000
d = 1000
a1 = np.random.randn(d0, d1).astype(np.float32)
a2 = np.random.randn(d1, d2).astype(np.float32)
a1 = np.random.randn(d, d).astype(np.float32) / math.sqrt(d)
a2 = np.random.randn(d, d).astype(np.float32) / math.sqrt(d)
device = "cuda"
# device = "cpu"
t1 =  torch.tensor(a1, device=device)
t2 =  torch.tensor(a2, device=device)
t = t1

In [13]:
import time
def f(t1, t2):
    s = torch.tensor(0.0, device=device)
    for i in range(1, t2.shape[0]):
        for j in range(1, t2.shape[1]):
            s+= torch.tensordot(t1[:i, :j], t2[:i, :j])
    t1[0][0] = s
    res = t1
    return res

for i in range(10):
    t0 = time.time()
    t = f(t, t2)
    dt = time.time() - t0
    print(f"{i} {1000 * dt:.2f}ms")
    # print(f"{1000 * dt:.2f}ms {t[0][0]}")
    


0 372634.03ms
1 366126.33ms
2 365221.94ms


KeyboardInterrupt: 

In [5]:
device = "cuda"
a = torch.tensor(2.0, requires_grad=True, device=device)
b = torch.tensor(3.0, device=device)

c = a * b
c.backward()

In [5]:
torch.backends.cpu.get_cpu_capability() 
torch.backends.cuda.matmul.allow_tf32
torch.backends.cudnn.is_available()
torch.backends.cudnn.allow_tf32
torch.backends.cuda.deterministic


AttributeError: module 'torch.backends.cuda' has no attribute 'deterministic'